In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from pyspark.ml import *
from pyspark.ml.classification import *

from pyspark.ml.feature import *
from pyspark.ml.param import *
from pyspark.ml.tuning import *
from pyspark.ml.evaluation import *
from pyspark.sql.types import DoubleType

from pyspark.sql.functions import rand 
from sklearn.metrics import classification_report
from time import time
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row

In [0]:
print("Running Spark Version %s" % (sc.version))

Running Spark Version 3.0.1

In [0]:
# sql(sqlContext,"set spark.sql.caseSensitive=true")
spark.conf.set('spark.sql.caseSensitive', "false")

In [0]:
df_Toys = spark.read.format("json").load("dbfs:/FileStore/shared_uploads/ckchancc@connect.ust.hk/Toys_and_Games_5.json")
df_Sports = spark.read.format("json").load("dbfs:/FileStore/shared_uploads/ckchancc@connect.ust.hk/Sports_and_Outdoors_5.json")
df_Video = spark.read.format("json").load("dbfs:/FileStore/shared_uploads/ckchancc@connect.ust.hk/Video_Games_5.json")


In [0]:
import functools 
def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

In [0]:
df = unionAll([df_Video, df_Sports, df_Toys])

In [0]:
df.show()

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
 asin| helpful|overall| reviewText| reviewTime| reviewerID| reviewerName| summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
0700099867| [8, 12]| 1.0|Installing the ga...| 07 9, 2012|A2HD75EMZR8QLN| 123|Pay to unlock con...| 1341792000|
0700099867| [0, 0]| 4.0|If you like rally...|06 30, 2013|A3UR8NLLY1ZHCX|Alejandro Henao "...| Good rally game| 1372550400|
0700099867| [0, 0]| 1.0|1st shipment rece...|06 28, 2014|A1INA0F5CWW3J4|Amazon Shopper "M...| Wrong key| 1403913600|
0700099867| [7, 10]| 3.0|I got this versio...|09 14, 2011|A1DLMTOTHQ4AST| ampgreen|awesome game, if ...| 1315958400|
0700099867| [2, 2]| 4.0|I had Dirt 2 on X...|06 14, 2011|A361M14PU2GUEG|Angry Ryan "Ryan ...| DIRT 3| 1308009600|
0700099867| [0, 0]| 4.0|Overall this is a...|05 11, 2013|A2UTRVO4FDCBH6| A.R.G.|Good racing game,...| 1368230400|
0700099867|[11, 13]| 5.0|Loved playing Dir...|08 14, 2011| AN3YYDZAS3O1Y| Bob|A step up from Di...| 1313280000|
0700099867| [1, 4]| 1.0|I can't tell you ...|11 24, 2012| AQTC623NCESZW| Chesty Puller|Crash 3 is correc...| 1353715200|
0700099867| [0, 1]| 4.0|I initially gave ...|11 14, 2012|A1QJJU33VNC4S7| D@rkFX|A great game ruin...| 1352851200|
0700099867| [1, 1]| 2.0|I still haven't f...| 02 8, 2014|A2JLT2WY0F2HVI| D. Sweetapple|Couldn't get this...| 1391817600|
0700099867| [1, 1]| 5.0|I'm not quite fin...|06 28, 2011|A38NXTZUFB1O2K| FiSH| Best in the series!| 1309219200|
0700099867| [0, 0]| 5.0|I have been playi...|10 19, 2011| ANW6EGY12V5XS| Henri Savin| A 5 stars winner!| 1318982400|
0700099867| [0, 1]| 5.0|Dirt 3 on DVDi co...|12 29, 2013| AHT34BRYFBFT1| hewimp| Cars| 1388275200|
0700099867| [0, 0]| 1.0|I bought this and...|06 30, 2014|A248LSBZT4P38V| Joseph R. Kennedy|It might have bee...| 1404086400|
0700099867| [0, 0]| 1.0|Crashed in Vista....|04 30, 2013| AFS6WERAP409A|Lisa Gamuciello "...|Don't waste your ...| 1367280000|
0700099867| [1, 1]| 1.0|This game was a r...|07 15, 2013|A14L115LBOB0A5| MotoRider42HC|Not as good as Di...| 1373846400|
0700099867| [1, 1]| 4.0|In today's game m...|03 19, 2012| A20DRRKAN5Z9Q| Moviedude|An overlooked gem...| 1332115200|
0700099867| [0, 0]| 3.0|This really is a ...|06 28, 2013|A2VYL51WQXC5KK| Old-Lumens "O-L"|Better than Dirt ...| 1372377600|
0700099867| [2, 5]| 1.0|This game is bug ...|12 26, 2011|A10AYT89XSCE46| Paulo Coelho|Colin McRae CRASH 3!| 1324857600|
0700099867| [3, 3]| 1.0|DiRT 2 was like t...|11 23, 2013|A29KT7UP7DLM1J| Peter Ostman|The first one was...| 1385164800|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
only showing top 20 rows

In [0]:
df.printSchema()

root
-- asin: string (nullable = true)
-- helpful: array (nullable = true)
 |-- element: long (containsNull = true)
-- overall: double (nullable = true)
-- reviewText: string (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- reviewerName: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: long (nullable = true)

In [0]:
df1 = df.select(df['reviewerID'],df['reviewText'],df['asin'],df['overall'].alias('label'))
df1=df1.dropna()
df1.show()

+--------------+--------------------+----------+-----+
 reviewerID| reviewText| asin|label|
+--------------+--------------------+----------+-----+
A2HD75EMZR8QLN|Installing the ga...|0700099867| 1.0|
A3UR8NLLY1ZHCX|If you like rally...|0700099867| 4.0|
A1INA0F5CWW3J4|1st shipment rece...|0700099867| 1.0|
A1DLMTOTHQ4AST|I got this versio...|0700099867| 3.0|
A361M14PU2GUEG|I had Dirt 2 on X...|0700099867| 4.0|
A2UTRVO4FDCBH6|Overall this is a...|0700099867| 4.0|
 AN3YYDZAS3O1Y|Loved playing Dir...|0700099867| 5.0|
 AQTC623NCESZW|I can't tell you ...|0700099867| 1.0|
A1QJJU33VNC4S7|I initially gave ...|0700099867| 4.0|
A2JLT2WY0F2HVI|I still haven't f...|0700099867| 2.0|
A38NXTZUFB1O2K|I'm not quite fin...|0700099867| 5.0|
 ANW6EGY12V5XS|I have been playi...|0700099867| 5.0|
 AHT34BRYFBFT1|Dirt 3 on DVDi co...|0700099867| 5.0|
A248LSBZT4P38V|I bought this and...|0700099867| 1.0|
 AFS6WERAP409A|Crashed in Vista....|0700099867| 1.0|
A14L115LBOB0A5|This game was a r...|0700099867| 1.0|
 A20DRRKAN5Z9Q|In today's game m...|0700099867| 4.0|
A2VYL51WQXC5KK|This really is a ...|0700099867| 3.0|
A10AYT89XSCE46|This game is bug ...|0700099867| 1.0|
A29KT7UP7DLM1J|DiRT 2 was like t...|0700099867| 1.0|
+--------------+--------------------+----------+-----+
only showing top 20 rows

In [0]:
df1.count()

Out[11]: 695714

In [0]:
df2 = df1.select(df1['reviewerID'],df1['asin'],df1['reviewText'],df1['label']).where(df1['label']!=3)

In [0]:
from pyspark.sql import functions as F
def assign_postive_negative_label(record):
    if record <= 2.0:
        return int(0)
    elif record >= 4.0:
        return int(1)

assign_label_udf = F.udf(assign_postive_negative_label)
df3 = df2.withColumn("label", assign_label_udf("label"))

In [0]:
df3.show()

+--------------+----------+--------------------+-----+
 reviewerID| asin| reviewText|label|
+--------------+----------+--------------------+-----+
A2HD75EMZR8QLN|0700099867|Installing the ga...| 0|
A3UR8NLLY1ZHCX|0700099867|If you like rally...| 1|
A1INA0F5CWW3J4|0700099867|1st shipment rece...| 0|
A361M14PU2GUEG|0700099867|I had Dirt 2 on X...| 1|
A2UTRVO4FDCBH6|0700099867|Overall this is a...| 1|
 AN3YYDZAS3O1Y|0700099867|Loved playing Dir...| 1|
 AQTC623NCESZW|0700099867|I can't tell you ...| 0|
A1QJJU33VNC4S7|0700099867|I initially gave ...| 1|
A2JLT2WY0F2HVI|0700099867|I still haven't f...| 0|
A38NXTZUFB1O2K|0700099867|I'm not quite fin...| 1|
 ANW6EGY12V5XS|0700099867|I have been playi...| 1|
 AHT34BRYFBFT1|0700099867|Dirt 3 on DVDi co...| 1|
A248LSBZT4P38V|0700099867|I bought this and...| 0|
 AFS6WERAP409A|0700099867|Crashed in Vista....| 0|
A14L115LBOB0A5|0700099867|This game was a r...| 0|
 A20DRRKAN5Z9Q|0700099867|In today's game m...| 1|
A10AYT89XSCE46|0700099867|This game is bug ...| 0|
A29KT7UP7DLM1J|0700099867|DiRT 2 was like t...| 0|
A15PIAQT55GNCA|0700099867|lot of people don...| 1|
A18YFCLL3GBD0T|0700099867|I would like give...| 0|
+--------------+----------+--------------------+-----+
only showing top 20 rows

In [0]:
from pyspark.sql.types import DoubleType
df3 = df3.withColumn("label", df3["label"].cast (DoubleType()))

In [0]:
df3.printSchema()

root
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewText: string (nullable = true)
-- label: double (nullable = true)

In [0]:
from pyspark.ml.feature import StopWordsRemover, RegexTokenizer
from pyspark.sql.functions import expr
import string

In [0]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [0]:
sw = StopWordsRemover().getStopWords()
punctuation = list(string.punctuation)
mylist = sw + punctuation
notremove=["not","none","nothing","nowhere","never","cannot",
           "cant","couldnt","except","hasnt","neither","no","nobody","nor","without"]
mylist = [x for x in mylist if x not in notremove]
list_keys=list(contractions.keys())
mylist = [x for x in mylist if x not in list_keys]

In [0]:
# convert the distinct labels in the input dataset to index values
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df3)

In [0]:
# tokenizer 
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\W")##'\w' remove none-word letters
df_tokenized = tokenizer.transform(df3)
df_tokenized.show()

+--------------+----------+--------------------+-----+--------------------+
 reviewerID| asin| reviewText|label| words|
+--------------+----------+--------------------+-----+--------------------+
A2HD75EMZR8QLN|0700099867|Installing the ga...| 0.0|[installing, the,...|
A3UR8NLLY1ZHCX|0700099867|If you like rally...| 1.0|[if, you, like, r...|
A1INA0F5CWW3J4|0700099867|1st shipment rece...| 0.0|[1st, shipment, r...|
A361M14PU2GUEG|0700099867|I had Dirt 2 on X...| 1.0|[i, had, dirt, 2,...|
A2UTRVO4FDCBH6|0700099867|Overall this is a...| 1.0|[overall, this, i...|
 AN3YYDZAS3O1Y|0700099867|Loved playing Dir...| 1.0|[loved, playing, ...|
 AQTC623NCESZW|0700099867|I can't tell you ...| 0.0|[i, can, t, tell,...|
A1QJJU33VNC4S7|0700099867|I initially gave ...| 1.0|[i, initially, ga...|
A2JLT2WY0F2HVI|0700099867|I still haven't f...| 0.0|[i, still, haven,...|
A38NXTZUFB1O2K|0700099867|I'm not quite fin...| 1.0|[i, m, not, quite...|
 ANW6EGY12V5XS|0700099867|I have been playi...| 1.0|[i, have, been, p...|
 AHT34BRYFBFT1|0700099867|Dirt 3 on DVDi co...| 1.0|[dirt, 3, on, dvd...|
A248LSBZT4P38V|0700099867|I bought this and...| 0.0|[i, bought, this,...|
 AFS6WERAP409A|0700099867|Crashed in Vista....| 0.0|[crashed, in, vis...|
A14L115LBOB0A5|0700099867|This game was a r...| 0.0|[this, game, was,...|
 A20DRRKAN5Z9Q|0700099867|In today's game m...| 1.0|[in, today, s, ga...|
A10AYT89XSCE46|0700099867|This game is bug ...| 0.0|[this, game, is, ...|
A29KT7UP7DLM1J|0700099867|DiRT 2 was like t...| 0.0|[dirt, 2, was, li...|
A15PIAQT55GNCA|0700099867|lot of people don...| 1.0|[lot, of, people,...|
A18YFCLL3GBD0T|0700099867|I would like give...| 0.0|[i, would, like, ...|
+--------------+----------+--------------------+-----+--------------------+
only showing top 20 rows

In [0]:
# remove stop words
remover = StopWordsRemover(inputCol='words', outputCol='cleaned_words', stopWords=mylist)
df_removed = remover.transform(df_tokenized)

In [0]:
# labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df3)
# tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\W")
# df_tokenized = tokenizer.transform(df3)
# remover = StopWordsRemover(inputCol='words', outputCol='cleaned_words', stopWords=mylist)
# df_removed = remover.transform(df_tokenized)
# # Convert to TF words vector
# hashingTF = HashingTF(inputCol="cleaned_words", outputCol="rawFeatures")
# df_TF = hashingTF.transform(df_removed)
# # Convert to TF*IDF words vector
# idf = IDF(inputCol="rawFeatures", outputCol="features")
# idfModel = idf.fit(df_TF)
# df_idf = idfModel.transform(df_TF)

In [0]:
# Convert to TF words vector
hashingTF = HashingTF(inputCol="cleaned_words", outputCol="rawFeatures")
df_TF = hashingTF.transform(df_removed)

# Convert to TF*IDF words vector
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(df_TF)
df_idf = idfModel.transform(df_TF)

for features_label in df_idf.select("features", "label").take(3):
    print(features_label)

Row(features=SparseVector(262144, {2410: 4.1634, 4874: 8.5365, 4978: 4.7626, 6034: 4.2879, 6355: 6.2995, 8254: 3.3721, 12524: 2.0337, 18640: 7.9415, 20263: 4.103, 26445: 3.9958, 27109: 8.4361, 31605: 7.3826, 32890: 5.0462, 33053: 3.98, 39239: 4.6342, 39510: 5.2525, 45161: 5.5955, 60433: 9.2378, 64415: 6.2348, 64841: 6.9805, 68685: 4.5022, 72762: 7.7842, 80649: 4.4558, 84685: 3.9764, 89833: 2.258, 92651: 2.585, 94266: 8.7043, 94896: 8.1954, 98773: 5.3622, 99855: 4.4269, 100510: 8.4659, 103723: 5.5802, 113673: 2.5909, 114031: 11.7393, 114598: 5.1704, 125752: 3.2537, 127702: 3.7112, 129422: 4.1639, 131264: 11.9624, 134125: 2.6225, 134304: 3.6145, 138895: 5.0503, 147962: 9.6111, 147994: 5.5072, 156356: 7.1708, 169300: 3.8026, 172164: 3.199, 174582: 4.7566, 184641: 4.8667, 185276: 4.8129, 186312: 1.7049, 196811: 24.6212, 198405: 6.972, 202517: 21.2732, 208258: 1.1988, 213760: 2.1415, 214179: 2.4436, 217669: 4.6696, 219389: 2.2584, 229264: 1.6248, 229914: 7.5436, 231821: 22.1554, 237532: 4.6151, 240840: 3.7277, 240900: 5.1394, 242066: 5.3877, 245919: 4.8517, 247840: 5.5701, 248039: 4.8436, 248090: 2.1599, 248179: 2.5879, 252722: 1.4084, 253382: 2.5667, 257750: 3.6638, 258147: 3.3888, 260199: 6.4944}), label=0.0)
Row(features=SparseVector(262144, {2410: 4.1634, 8145: 4.5316, 23087: 1.7817, 45190: 3.4372, 62312: 5.4698, 92232: 5.8757, 110510: 3.1908, 131250: 3.3748, 138895: 2.5251, 166027: 2.2666, 174966: 1.8493, 181321: 2.4165, 184195: 4.1875, 194612: 7.2757, 198197: 3.8847, 208258: 1.1988, 214982: 3.8604, 217187: 13.8771, 217461: 12.6873, 217680: 6.0163, 228060: 6.2695, 230876: 5.8666, 231821: 14.7703, 233967: 2.516, 252722: 1.4084}), label=1.0)
Row(features=SparseVector(262144, {21823: 1.1615, 22370: 3.0758, 38698: 3.5833, 52351: 3.2595, 62363: 10.56, 64465: 4.6635, 77073: 4.2402, 100510: 8.4659, 111986: 14.2843, 113432: 1.3939, 116996: 3.1596, 122367: 5.2049, 124217: 5.5171, 131640: 12.407, 138895: 2.5251, 155882: 6.0819, 162155: 4.5189, 168976: 3.1059, 170193: 5.1899, 177873: 3.8482, 189113: 4.3417, 201440: 3.5764, 212846: 4.4996, 225898: 4.2279, 240958: 5.1814, 243875: 5.0071, 245730: 7.4994}), label=0.0)

In [0]:
# Split data aproximately into training (80%) and test (20%)
(train, test)=df3.randomSplit([0.8,0.2], seed = 0)

# Cache the train and test data in-memory
train = train.cache()
test = test.cache()
print ('Sample number in the train set : {}'.format(train.count()))
print ('Sample number in the test set : {}'.format(test.count()))
train.groupby('label').count().show()

Sample number in the train set : 501794
Sample number in the test set : 125217
+-----+------+
label| count|
+-----+------+
 0.0| 46904|
 1.0|454890|
+-----+------+

In [0]:
#NaiveBayes
def grid_search(p1,p2):
    nb = NaiveBayes()
    pipeline = Pipeline(stages=[labelIndexer,tokenizer, remover, hashingTF, idfModel, nb])
  
    #Create ParamGrid for Cross Validation
    paramGrid = (ParamGridBuilder()
                 .addGrid(hashingTF.numFeatures, [p1])
                 .addGrid(nb.smoothing, [p2])
                 .build())
    evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=4)
    
    ########  Run cross-validation, and choose the best set of parameters.
    cvModel = crossval.fit(train)
    # average cross-validation accuracy metric/s on all folds
    average_score = cvModel.avgMetrics
    print('average cross-validation accuracy = {}'.format(average_score[0]))
    return average_score[0]

In [0]:
score=0.0
for p1 in [35000,40000,55000]:
    for p2 in [0.8,0.9,1.0]:
      t0 = time()
      print('(numFeatures,smoothing)=({},{})'.format(p1,p2))
      average_score=grid_search(p1,p2)
      tt = time() - t0
      print("Classifier trained in {} seconds".format(round(tt,3)))
      if average_score > score:
        print('################ Best score ######################')
        params=(p1,p2)
        score=average_score
print('Best score is {} at params ={}'.format(score, params))

In [0]:
def Data_modeling(train, test, pipeline, paramGrid):
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=4)
    
    ########  Run cross-validation, and choose the best set of parameters.
    cvModel = crossval.fit(train)
    
    ########  Make predictions on on the test data
    prediction = cvModel.transform(test)
    average_score = cvModel.avgMetrics
    print('average cross-validation accuracy = {}'.format(average_score[0]))
    ######## Calculate accuracy of the prediction of the test data
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
    accuracy_score=evaluator.evaluate(prediction)
    # another way to calculate accuracy 
    #correct=prediction.filter(prediction['label']== prediction['prediction']).select("label","prediction")
    #accuracy_score = correct.count() / float(test.count())  
    print('Accuracy in the test data = {}'.format(accuracy_score))
    
    ######## calculate F1 score of the prediction of the test data
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
    f1_score=evaluator.evaluate(prediction)
    print('F1 score in the test data = {}'.format(f1_score)) 
    # Calculate area under ROC for the prediction of the test data
    #evaluator = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")
    #ROC_score=evaluator.evaluate(prediction)
    #print 'areaUnderROC in the test data = {}'.format(ROC_score)
    
    ######## Print classification_report
    prediction_and_labels=prediction.select("label","prediction")
    y_true = []
    y_pred = []
    for x in prediction_and_labels.collect():
        xx = list(x)
        try:
            tt = int(xx[1])
            pp = int(xx[0])
            y_true.append(tt)
            y_pred.append(pp)
        except:
            continue

    target_names = ['0', '1']
    print(classification_report(y_true, y_pred, target_names=target_names))
    return 

In [0]:
# trained by a Naïve Bayes 
nb = NaiveBayes()
# Build a pipeline
pipeline = Pipeline(stages=[labelIndexer,tokenizer, remover, hashingTF, idfModel, nb])
# Create ParamGrid for Cross Validation 
paramGrid = (ParamGridBuilder()
             .addGrid(hashingTF.numFeatures, [55000])
             .addGrid(nb.smoothing, [1.0])
             .build())
# Execute 4-folds cross validation for hyperparameter tuning, model prediction and model evaluation.
Data_modeling(train, test, pipeline, paramGrid)


average cross-validation accuracy = 0.8690500842360542
Accuracy in the test data = 0.8678454203502719
F1 score in the test data = 0.8846035881265868
 precision recall f1-score support

 0 0.73 0.39 0.51 22074
 1 0.88 0.97 0.92 103143

 accuracy 0.87 125217
 macro avg 0.81 0.68 0.72 125217
weighted avg 0.86 0.87 0.85 125217

In [0]:
#LogisticRegression
def grid_search(p1,p2,p3,p4):
    lr = LogisticRegression()
    pipeline = Pipeline(stages=[labelIndexer,tokenizer, remover, hashingTF, idfModel, lr])
  
    #Create ParamGrid for Cross Validation
    paramGrid = (ParamGridBuilder()
                 .addGrid(hashingTF.numFeatures, [p1])
                 .addGrid(lr.regParam, [p2])
                 .addGrid(lr.elasticNetParam, [p3])
                 .addGrid(lr.maxIter, [p4])
                 .build())
    evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=4)
    
    ########  Run cross-validation, and choose the best set of parameters.
    cvModel = crossval.fit(train)
    # average cross-validation accuracy metric/s on all folds
    average_score = cvModel.avgMetrics
    print('average cross-validation accuracy = {}'.format(average_score[0]))
    return average_score[0]

In [0]:
score=0.0
for p1 in [45000,50000,55000]:
    for p2 in [0.09,0.10,0.11]:
        for p3 in [0.09,0.10,0.11]:
            for p4 in [9,10,11]:
                t0 = time()
                print('(numFeatures,regParam,elasticNetParam,maxIter)=({},{},{},{})'.format(p1,p2,p3,p4))
                average_score=grid_search(p1,p2,p3,p4)
                tt = time() - t0
                print("Classifier trained in {} seconds".format(round(tt,3)))
                if average_score > score:
                    print('################ Best score ######################')
                    params=(p1,p2,p3,p4)
                    score=average_score
print('Best score is {} at params ={}'.format(score, params))

In [0]:
# trained by a logistic regression 
lr = LogisticRegression()
# Build a pipeline
pipeline = Pipeline(stages=[labelIndexer,tokenizer, remover, hashingTF, idfModel, lr])

# Create ParamGrid for Cross Validation 
paramGrid = (ParamGridBuilder()
             .addGrid(hashingTF.numFeatures, [55000])
             .addGrid(lr.regParam, [0.11])
             .addGrid(lr.elasticNetParam, [0.11])
             .addGrid(lr.maxIter, [11])
             .build())
# Execute 4-folds cross validation for hyperparameter tuning, model prediction and model evaluation.
Data_modeling(train, test, pipeline, paramGrid)


average cross-validation accuracy = 0.9083795580505375
Accuracy in the test data = 0.9070972791234417
F1 score in the test data = 0.8660472742735321
 precision recall f1-score support

 0 0.03 0.78 0.05 419
 1 1.00 0.91 0.95 124798

 accuracy 0.91 125217
 macro avg 0.51 0.84 0.50 125217
weighted avg 1.00 0.91 0.95 125217

In [0]:
#Decision Tree
def grid_search(p1,p2,p3):
    # trained by a Decision Tree 
    dt = DecisionTreeClassifier(labelCol="label",impurity="entropy")
    pipeline = Pipeline(stages=[labelIndexer,tokenizer, remover, hashingTF, idfModel, dt])
  
    #Create ParamGrid for Cross Validation
    paramGrid = (ParamGridBuilder()
                 .addGrid(hashingTF.numFeatures, [p1])
                 .addGrid(dt.maxDepth, [p2])
                 .addGrid(dt.minInstancesPerNode, [p3])
                 .build())
    evaluator = MulticlassClassificationEvaluator(labelCol="Label", predictionCol="prediction", metricName="accuracy")
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=4)
    
    ########  Run cross-validation, and choose the best set of parameters.
    cvModel = crossval.fit(train)
    # average cross-validation accuracy metric/s on all folds
    average_score = cvModel.avgMetrics
    print('average cross-validation accuracy = {}'.format(average_score[0]))
    return average_score[0]

In [0]:
score=0.0
# for p1 in [65000,70000,75000]:
#     for p2 in [23,24,25,26,27]:
#         for p3 in [3,4,5]:
for p1 in [65000,70000,75000]:
    for p2 in [12,13,14]:
        for p3 in [3,4]:
          t0 = time()
          print('(numFeatures,maxDepth,minInstancesPerNode)=({},{},{})'.format(p1,p2,p3))
          average_score=grid_search(p1,p2,p3)
          tt = time() - t0
          print("Classifi er trained in {} seconds".format(round(tt,3)))
          if average_score > score:
            print('################ Best score ######################')
            params=(p1,p2,p3)
            score=average_score
print('Best score is {} at params ={}'.format(score, params))

(numFeatures,maxDepth,minInstancesPerNode)=(65000,12,3)
average cross-validation accuracy = 0.9118916761315732
Classifi er trained in 7690.869 seconds
################ Best score ######################
(numFeatures,maxDepth,minInstancesPerNode)=(65000,12,4)
average cross-validation accuracy = 0.9120474802483308
Classifi er trained in 7652.926 seconds
################ Best score ######################
(numFeatures,maxDepth,minInstancesPerNode)=(65000,13,3)
average cross-validation accuracy = 0.9119318764113887
Classifi er trained in 8408.561 seconds
(numFeatures,maxDepth,minInstancesPerNode)=(65000,13,4)
average cross-validation accuracy = 0.9120417350677993
Classifi er trained in 8551.208 seconds
(numFeatures,maxDepth,minInstancesPerNode)=(65000,14,3)

In [0]:
# trained by a Decision Tree 
dt = DecisionTreeClassifier(labelCol="label",impurity="entropy")
# Build a pipeline
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idfModel, dt])
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(hashingTF.numFeatures, [65000])
             .addGrid(dt.maxDepth, [12])
             .addGrid(dt.minInstancesPerNode, [4])
             .build())
# Execute 4-folds cross validation for hyperparameter tuning, model prediction and model evaluation.
Data_modeling(train, test, pipeline, paramGrid)

average cross-validation accuracy = 0.911850000661897
Accuracy in the test data = 0.9124959071052653
F1 score in the test data = 0.88868899744373
 precision recall f1-score support

 0 0.17 0.64 0.27 3143
 1 0.99 0.92 0.95 122074

 accuracy 0.91 125217
 macro avg 0.58 0.78 0.61 125217
weighted avg 0.97 0.91 0.94 125217

In [0]:
#Random Forest
def grid_search(p1,p2,p3,p4):
    rf = RandomForestClassifier(labelCol="label",impurity="entropy", seed=5043)
    pipeline = Pipeline(stages=[labelIndexer,tokenizer, remover, hashingTF, idfModel, rf])
  
    #Create ParamGrid for Cross Validation
    paramGrid = (ParamGridBuilder()
                 .addGrid(hashingTF.numFeatures, [p1])
                 .addGrid(rf.numTrees, [p2])
                 .addGrid(rf.maxDepth, [p3])
                 .addGrid(rf.minInstancesPerNode, [p4])
                 .build())
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=4)
    
    ########  Run cross-validation, and choose the best set of parameters.
    cvModel = crossval.fit(train)
    # average cross-validation accuracy metric/s on all folds
    average_score = cvModel.avgMetrics
    print('average cross-validation accuracy = {}'.format(average_score[0]))
    return average_score[0]

In [0]:
score=0.0
# for p1 in [45000,50000,55000]:
#     for p2 in [30,31,32]:
#         for p3 in [28,29,30]:
for p1 in [45000,50000,55000]:
    for p2 in [12,13,14]:
        for p3 in [12,13,14]:
            for p4 in [1,2]:
                t0 = time()
                print('(numFeatures,numTrees,maxDepth,minInstancesPerNode)=({},{},{},{})'.format(p1,p2,p3,p4))
                average_score=grid_search(p1,p2,p3,p4)
                tt = time() - t0
                print("Classifier trained in {} seconds".format(round(tt,3)))
                if average_score > score:
                  print('################ Best score ######################')
                  params=(p1,p2,p3,p4)
                  score=average_score
print('Best score is {} at params ={}'.format(score, params))

(numFeatures,numTrees,maxDepth,minInstancesPerNode)=(50000,12,12,1)
average cross-validation accuracy = 0.9065320640826997
Classifier trained in 3442.122 seconds
################ Best score ######################
(numFeatures,numTrees,maxDepth,minInstancesPerNode)=(50000,12,12,2)

In [0]:
rf = RandomForestClassifier(labelCol="label",impurity="entropy", seed=5043)
# Build a pipeline
pipeline = Pipeline(stages=[labelIndexer,tokenizer, remover, hashingTF, idfModel, rf])

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(hashingTF.numFeatures, [45000])
             .addGrid(rf.numTrees, [10])
             .addGrid(rf.maxDepth, [12])
             .addGrid(rf.minInstancesPerNode, [1])
             .build())
# Execute 4-folds cross validation for hyperparameter tuning, model prediction and model evaluation.
Data_modeling(train, test, pipeline, paramGrid)

average cross-validation accuracy = 0.9065460199531977
Accuracy in the test data = 0.9052524816917831
F1 score in the test data = 0.8602506088727122
 precision recall f1-score support

 0 0.00 1.00 0.00 2
 1 1.00 0.91 0.95 125215

 accuracy 0.91 125217
 macro avg 0.50 0.95 0.48 125217
weighted avg 1.00 0.91 0.95 125217

In [0]:
#Gradient Boosted Tree
def grid_search(p1,p2,p3,p4):
    gbt = GBTClassifier(labelCol="label")
    pipeline = Pipeline(stages=[labelIndexer,tokenizer, remover, hashingTF, idfModel, gbt])
  
    #Create ParamGrid for Cross Validation
    paramGrid = (ParamGridBuilder()
                 .addGrid(hashingTF.numFeatures, [p1])
                 .addGrid(gbt.maxIter, [p2]) #(default: 20)
                 .addGrid(gbt.maxDepth, [p3])
                 .addGrid(gbt.minInstancesPerNode, [p4])
                 .build())
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=4)
    
    ########  Run cross-validation, and choose the best set of parameters.
    cvModel = crossval.fit(train)
    # average cross-validation accuracy metric/s on all folds
    average_score = cvModel.avgMetrics
    print('average cross-validation accuracy = {}'.format(average_score[0]))
    return average_score[0]


In [0]:
score=0.0
# for p1 in [60000]:
#     for p2 in [25,26,27]:
#         for p3 in [18,19,20]:
for p1 in [60000]:
    for p2 in [15,16,17]:
        for p3 in [12,13,14]:
            for p4 in [2]: 
                t0 = time()
                print('(numFeatures,maxIter,maxDepth,minInstancesPerNode)=({},{},{},{})'.format(p1,p2,p3,p4))
                average_score=grid_search(p1,p2,p3,p4)
                tt = time() - t0
                print("Classifier trained in {} seconds".format(round(tt,3)))
                if average_score > score:
                  print('################ Best score ######################')
                  params=(p1,p2,p3,p4)
                  score=average_score
print('Best score is {} at params ={}'.format(score, params))

(numFeatures,maxIter,maxDepth,minInstancesPerNode)=(60000,15,12,2)
average cross-validation accuracy = 0.9187180914947017
Classifier trained in 99132.029 seconds
################ Best score ######################
(numFeatures,maxIter,maxDepth,minInstancesPerNode)=(60000,15,13,2)

In [0]:
# trained by a Gradient Boosted Tree 
gbt = GBTClassifier(labelCol="label")
# Build a pipeline
pipeline = Pipeline(stages=[labelIndexer,tokenizer, remover, hashingTF, idfModel, gbt])
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(hashingTF.numFeatures, [60000])
             .addGrid(gbt.maxIter, [15]) #(default: 20)
             .addGrid(gbt.maxDepth, [12])
             .addGrid(gbt.minInstancesPerNode, [1])
             .build())
# Execute 4-folds cross validation for hyperparameter tuning, model prediction and model evaluation.
Data_modeling(train, test, pipeline, paramGrid)

In [0]:
#Summary
import matplotlib.pyplot as plt
classifier_names=['Naive_Bayes', 'Logistic_Regression', 'Random_Forest','Decision_Tree', 'Gradient_Boosted_Tree']
time=[102.438,154.102,7652.926,3442.122,99132.029]
accuracy=[0.867845420350271,0.907842824055586,0.90115483,0.912495907105265,0.918]



fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(18,5), facecolor='white')
ax[0].barh(np.arange(0, 5),time)
ax[0].set_yticks(np.arange(0.5, 5.5))
ax[0].set_yticklabels(classifier_names)
ax[0].grid(color='b', linestyle='--', linewidth=1)
ax[0].set_title('Model training time')
ax[0].set_xlabel('Time (sec)')
ax[0].set_xscale('log')

ax[1].barh(range(0, len(classifier_names)),accuracy)
ax[1].set_xlim([0.75,0.85])
ax[1].set_title('Model evaluation on the test set')
ax[1].set_xlabel('Accuracy')
ax[1].set_yticklabels([])
ax[1].grid(color='b', linestyle='--', linewidth=1)
display(fig)